In [1]:
#current working code with NA filtered out. 

In [2]:
import xlwings as xw
import requests
import pandas as pd

wb = xw.Book('Richspread.xlsx')       
ticker_sht    = wb.sheets['Ticker']
annual_sht    = wb.sheets['Annual']
quarterly_sht = wb.sheets['Quarterly']

symbol  = ticker_sht.range('B2').value    
api_key = ''              

ov_url      = (
    f'https://www.alphavantage.co/query'
    f'?function=OVERVIEW'
    f'&symbol={symbol}'
    f'&apikey={api_key}'
)
overview     = requests.get(ov_url).json()
description  = overview.get('Description', 'No description available')

ticker_sht.range('A3').value = 'Description'
ticker_sht.range('A4').value = description

def fetch_df(function_name: str, report_type: str) -> pd.DataFrame:
    url = (
        f'https://www.alphavantage.co/query'
        f'?function={function_name}'
        f'&symbol={symbol}'
        f'&apikey={api_key}'
    )
    payload = requests.get(url).json()
    if 'Note' in payload or 'Error Message' in payload:
        print(f"⚠️ {function_name} {report_type}:",
              payload.get('Note') or payload.get('Error Message'))
        return pd.DataFrame()
    return pd.json_normalize(payload.get(report_type, []))

inc_ann = fetch_df('INCOME_STATEMENT', 'annualReports')
inc_qtr = fetch_df('INCOME_STATEMENT', 'quarterlyReports')
bs_ann  = fetch_df('BALANCE_SHEET',    'annualReports')
bs_qtr  = fetch_df('BALANCE_SHEET',    'quarterlyReports')
cf_ann  = fetch_df('CASH_FLOW',        'annualReports')
cf_qtr  = fetch_df('CASH_FLOW',        'quarterlyReports')

def clean_df(df):
    df.replace(to_replace=r'(?i)^none$', value=0, regex=True, inplace=True)
    df.fillna(0, inplace=True)
    return df

# =============================
# ANNUAL SHEET
# =============================
# Clear old data (adjust columns as needed)
annual_sht.range('A2:AZ44').clear_contents()   # Income Statement – Annual
annual_sht.range('A45:AZ89').clear_contents()  # Balance Sheet – Annual
annual_sht.range('A90:AZ150').clear_contents() # Cash Flow – Annual

# Annual Income Statement at A2
annual_sht.range('A2').value = 'Income Statement – Annual'
if not inc_ann.empty:
    inc_ann = clean_df(inc_ann)
    annual_sht.range('A3').options(index=False).value = inc_ann
else:
    annual_sht.range('A3').value = 'No data returned'

# Annual Balance Sheet at A45
annual_sht.range('A45').value = 'Balance Sheet – Annual'
if not bs_ann.empty:
    bs_ann = clean_df(bs_ann)
    annual_sht.range('A46').options(index=False).value = bs_ann
else:
    annual_sht.range('A46').value = 'No data returned'

# Annual Cash Flow at A90
annual_sht.range('A90').value = 'Cash Flow – Annual'
if not cf_ann.empty:
    cf_ann = clean_df(cf_ann)
    annual_sht.range('A91').options(index=False).value = cf_ann
else:
    annual_sht.range('A91').value = 'No data returned'

# =============================
# QUARTERLY SHEET
# =============================
quarterly_sht.range('A2:AZ99').clear_contents()    # Income Statement – Quarterly
quarterly_sht.range('A100:AZ199').clear_contents() # Balance Sheet – Quarterly
quarterly_sht.range('A200:AZ260').clear_contents() # Cash Flow – Quarterly

# Quarterly Income Statement at A2
quarterly_sht.range('A2').value = 'Income Statement – Quarterly'
if not inc_qtr.empty:
    inc_qtr = clean_df(inc_qtr)
    quarterly_sht.range('A3').options(index=False).value = inc_qtr
else:
    quarterly_sht.range('A3').value = 'No data returned'

# Quarterly Balance Sheet at A100
quarterly_sht.range('A100').value = 'Balance Sheet – Quarterly'
if not bs_qtr.empty:
    bs_qtr = clean_df(bs_qtr)
    quarterly_sht.range('A101').options(index=False).value = bs_qtr
else:
    quarterly_sht.range('A101').value = 'No data returned'

# Quarterly Cash Flow at A200
quarterly_sht.range('A200').value = 'Cash Flow – Quarterly'
if not cf_qtr.empty:
    cf_qtr = clean_df(cf_qtr)
    quarterly_sht.range('A201').options(index=False).value = cf_qtr
else:
    quarterly_sht.range('A201').value = 'No data returned'

KeyboardInterrupt: 

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xlwings as xw
import tempfile
import os

def calculate_rsi(prices, window=14):
    delta = prices.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.rolling(window=window, min_periods=window).mean()
    roll_down = down.rolling(window=window, min_periods=window).mean()
    rs = roll_up / roll_down
    rsi = 100 - (100 / (1 + rs))
    return rsi

def ohlc_plot_candles_with_rsi(
    df,
    window=None,
    figsize=(18, 8),
    title='Candlestick Chart',
    bull_color='green',
    bear_color='red',
    doji_color='black',
    wick_color='black',
    linewidth_body=2,
    linewidth_wick=1,
    show_grid=True,
    xtick_interval=30,
    background_color='white',
    rsi_window=14,
    save_path=None
):
    if isinstance(df.columns, pd.MultiIndex):
        raise ValueError("MultiIndex columns detected; please select a single ticker's columns.")
    else:
        sample = df.copy()
    if window is not None and window <= len(sample):
        sample = sample.iloc[-window:].copy()
    sample['RSI'] = calculate_rsi(sample['Close'], window=rsi_window)
    fig, (ax1, ax2) = plt.subplots(
        2, 1,
        figsize=figsize,
        sharex=True,
        gridspec_kw={'height_ratios': [3, 1]}
    )
    fig.patch.set_facecolor('white')
    fig.suptitle(title, color='black')
    ax1.set_facecolor(background_color)
    ax2.set_facecolor(background_color)
    for i in range(len(sample)):
        ax1.vlines(
            x=i,
            ymin=float(sample.iloc[i]['Low']),
            ymax=float(sample.iloc[i]['High']),
            color=wick_color,
            linewidth=linewidth_wick
        )
        open_ = float(sample.iloc[i]['Open'])
        close_ = float(sample.iloc[i]['Close'])
        if close_ > open_:
            ax1.vlines(
                x=i, ymin=open_, ymax=close_,
                color=bull_color, linewidth=linewidth_body)
        elif close_ < open_:
            ax1.vlines(
                x=i, ymin=close_, ymax=open_,
                color=bear_color, linewidth=linewidth_body)
        else:
            ax1.vlines(
                x=i, ymin=close_, ymax=open_+1e-6,
                color=doji_color, linewidth=linewidth_body)
    idxs = range(len(sample))
    labels = [sample.index[i].strftime('%Y-%m-%d') for i in idxs]
    show_idx = [i for i in idxs if i % xtick_interval == 0 or i == len(sample)-1]
    ax2.set_xticks(show_idx)
    ax2.set_xticklabels([labels[i] for i in show_idx], rotation=45, fontsize=10)
    ax1.set_ylabel("Price")
    if show_grid:
        ax1.grid(axis='y', color='gray', alpha=0.3)
    ax2.plot(range(len(sample)), sample['RSI'], color='purple', label=f'RSI ({rsi_window})')
    ax2.axhline(70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
    ax2.axhline(30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
    ax2.set_ylabel('RSI')
    ax2.set_xlabel("Date")
    ax2.set_ylim(0, 100)
    ax2.legend(loc='upper left', fontsize=10)
    if show_grid:
        ax2.grid(axis='y', color='gray', alpha=0.3)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    if save_path is not None:
        plt.savefig(save_path, bbox_inches='tight', facecolor=fig.get_facecolor())
        plt.close(fig)
    else:
        plt.show()

# ---- MAIN SCRIPT ----
book = xw.Book('Richspread.xlsx')
sht = book.sheets['Funda']
ticker = sht.range('B2').value
Title = f"{ticker} Candlestick Chart with RSI"
start_date = sht.range('Y5').value
end_date = sht.range('Y6').value

with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
    chart_path = tmpfile.name

df = yf.download(ticker, start=start_date, end=end_date)[['Open', 'High', 'Low', 'Close']]
if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[0] for col in df.columns]

ohlc_plot_candles_with_rsi(
    df,
    window=None,
    title=Title,
    xtick_interval=40,
    save_path=chart_path
)

Funda = book.sheets['Funda']
Funda.range('AA3').value = Title
Funda.pictures.add(chart_path, name=Title, update=True,
                   left=Funda.range('AA3').left,
                   top=Funda.range('AA3').top)

try:
    os.remove(chart_path)
except Exception:
    pass